#### This script implements the following steps for updating the inputs to a unit process (e.g., swapping electricity providers, revising amount of chemical use) 

- Identify the exchanges of interest (e.g., providers of electricity, raw materials) that contribute to the uncertainty of a unit process (e.g., flat glass production) at a given "layer" (e.g., foreground layer, immediate upstream layer of a material production, further upstream layers)
- Update the amount of these exchanges: by having an additional input parameter (e.g., update_amt) and adding a few more lines of code in recursion function
- Swapping providers: by having an input argument (a dictionary of {orig_provider: new_provider} ) and corresponding lines of code in recursion function


#### import libraries

In [316]:
import brightway2 as bw
from bw2data import get_activity
from collections import defaultdict
import os
import math
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### 1. Create functions

#### 1.1 create the recursion function

In [317]:
def recur_dig(activity,amount,search_list,depth,glo_store_dict,glo_relation_dict,parent,parent_amt,**kwargs):
    """This function applies recursion to dig out the input (exc.input) of TECHNOSPHERE exchanges of interest 
        
    [caution] the same input (e.g., same electricity provider for "electricity, medium voltage, RoW") used for
            eletric chiller and for electrocatalytic reactor) will be added to the same recursion lvl, b/c 'glo_store_dict(list)'
            accordingly, another function to aggregate the values of the same iput under one recursion lvl is needed
    ** Input params:
     * search_list: [(activity_name, location), ..., (activity_name, location) ]
     * parent: a string indicates the activity (in the previous layer) to which the current activity is supplying input
    ** the higher the depth value, to closer the it is to the "foreground" of a LCA model
     * typically the key of foreground starts with "depth - 1" (as the search list will likely not contain 
     * the act for the reference product e.g., 'flat glass production, uncoated | RoW' for 'flat glass' itself) 
    """
    
    ## parse activity
    if isinstance(activity, tuple):
        activity = get_activity(activity) #the activity tuple should be (database,code): seems only necessary for initial input
    
    ## record findings
    for (search_name, search_loc) in search_list:
        if search_name.lower() in activity['name'].lower() and search_loc.lower() in activity['location'].lower():
            # add to result storage dict
            glo_store_dict[depth].append({activity['name']+" | "+activity['location']:amount}) #add ['location'] as  unique label']
            # add relationship to the relationship storage dict
            glo_relation_dict[depth].append(activity['name']+" | "+activity['location']+" --> "+parent+" | "+str(parent_amt))
        

    ##Recursion step
    if depth>0:
        # reset parent activity to current activity
        parent = activity['name'] + " | " + activity['location']
        
        for exc in activity.technosphere():
            recur_dig(exc.input,exc['amount']*amount,search_list,depth-1,glo_store_dict,
                      glo_relation_dict, parent, parent_amt=amount) #[March 11, 2021] stopped using'abs(amount)' to adjust the amount of exchange



#### 1.2 create the aggregation function (for results generated by recursion function)

In [318]:
def recur_agg(recur_result_dict,agg_by_name_only=False,**kwargs):
    """This function aggregates the values of the same input of exchange (e.g., same electricity provider) 
    within one recursion lvl
    
    [caution] 'agg_by_name_only=True' refers to the situation when it is preferred to combine the input of echanges by name only, 
    regardless of other attributes (e.g., aggregate the amount of all providers of 'electricty, medium voltage', regardless of locations)
    
    """
    
    ##initiate an output dict
    recur_result_agg_dict=defaultdict(list)

    ##aggregate the activities
    for lvl, lst in recur_result_dict.items():
        #initiate a temp dict(list)
        temp_dict_list=defaultdict(list)
        for d in lst:
            for k,v in d.items():
                #check if aggregate based on name only
                if agg_by_name_only:
                    #only take the first part of the key: e.g., market for electricity, medium voltage from 'market for electricity, medium voltage_AU'
                    k=k.split('|')[0].strip()
                temp_dict_list[k].append(v)
        for k2,lst2 in temp_dict_list.items():
            recur_result_agg_dict[lvl].append({k2:sum(lst2)})
    
    return recur_result_agg_dict
    
    
    

#### 1.3 Create the subtraction function
- calculate the LCA results of a base activity (e.g, "flat glass production, uncoated, RoW")
- run LCA of the inputs of interest (e.g., calculate GWP for 0.5 kWh of "market for electricity, medium voltage, RoW" from 1st layer); aggregate the LCIA results and subtract it from that of the baseline activity—> return 'interm_lcia_results'

In [319]:
def subt_lcia(base_act, inputs_to_subt_dict, lcia_method_lst, UUID_only_flg):
    """ 
    - baseline_act: a nested tuple of (('db',base act_code), amt); e.g., (('ecoinvent 3.5 cutoff',flat_glass_uncoated_RoW['code']), 1)
    - inputs_to_subt_dict: a dict of aggregated (at each layer) inputs generated from recursion algrithm;
        e.g., {'layer0': [{'market for electricity, medium voltage_uuid':0.5}, ..., {'limestone, packed_uuid'':1.1}],
                ...
               'layer5': [{'market for electricity, medium voltage_uuid'':0.7}, ..., {'hydrogen production, steam reforming_uuid'':0.1}]
                }
        OR
        e.g., {'layer0': [{'uuid':0.5}, ..., {'uuid':1.1}],
                ...
               'layer5': [{'uuid':0.7}, ..., {'uuid':0.1}]
                }
    """
    
    
    ## prepare output variables
    output_lcia_results_dict_baseline={} #{'lcia_method1': lcia_result1, 'lcia_method2': lcia_result2, ...}
    output_lcia_results_dict_interm={} #{'lcia_method1': lcia_result1, 'lcia_method2': lcia_result2, ...}
    
    ## run LCA calculations
    for method in lcia_method_lst:
        # calculate the lcia results for baseline act first
        lca=bw.LCA({base_act[0]:base_act[1]},method)
        lca.lci()
        lca.lcia()
        output_lcia_results_dict_baseline[method]=lca.score       
        
        # calculate the lcia results for each input of interest and aggregate them into one result per impact category
        # create a temp list to store the lcia results for one method
        tmp_lcia_results_one_method=[]
        for lst_of_each_layer in inputs_to_subt_dict.values():
            for dict_ in lst_of_each_layer:
                for k,v in dict_.items():
                    # check the format to the subdict, if it is in {'uuid': amt}, then use the following procedure:
                    if UUID_only_flg == True:
                        tmp_FU = (base_act[0][0], k)
                    else:   
                        # create a temp tuple for ('db','code')
                        tmp_FU=(base_act[0][0], k.split('_')[1].strip()) #base_act[0] is a tuple of ('db',baseline act_code)
                    lca=bw.LCA({tmp_FU:v}, method)
                    lca.lci()
                    lca.lcia()
                    tmp_lcia_results_one_method.append(lca.score)
        # sum up the 'tmp_lcia_results_one_method' list, then subtract it from the base result
        output_lcia_results_dict_interm[method]=output_lcia_results_dict_baseline[method]-sum(tmp_lcia_results_one_method)

    return output_lcia_results_dict_baseline,output_lcia_results_dict_interm
    

#### 1.4 Create the update function 
- Similarly, run LCA of new inputs (a different/old input with a specific amount), aggregate the LCIA results  and add it back to the 'interm_lcia_results'—>return 'updated_lcia_results'

In [320]:
def update_lcia(interm_results, update_input_dict):
    """ 
    -interm_results: a dict of {'lcia_method1': interm_result1, ..., 'lcia_methodX': interm_resultX}
    -update_input_dict: a dict of {('db', new_input_code1'): amt1, ('db',new_input_code2'):amt2, ...}
    """

    ## prepare output variables
    output_updated_results={} #a dict of {'lcia_method1': updatedresult1, ..., 'lcia_methodX': updated_resultX}
    
    ## calculate the lcia results of the new inputs
    for method, result in interm_results.items():
        # create a temp list to store the result of each new input
        tmp_lcia_results_one_method=[]
        for layer,update_lst in update_input_dict.items():
            for sub_dict in update_lst:
                FU, amt = list(sub_dict.keys())[0], list(sub_dict.values())[0] # FU example: (db_name, mkt_group_elect_medium_vol_US_MRO[code])
                lca=bw.LCA({FU:amt}, method)
                lca.lci()
                lca.lcia()            
                tmp_lcia_results_one_method.append(lca.score)
        
        # sum up the 'tmp_lcia_results_one_method', then add it to the interm results
        output_updated_results[method]=interm_results[method]+sum(tmp_lcia_results_one_method)
        
    return  output_updated_results

#### 1.5 create uncertainty factor calculation function for INDIVIDUAL instance of variation
- a function to calculate the uncertainty factor of an instance of variation under a given uncertainty group: e.g., "change of electricity provider" in the uncertainty group "supplychain"


In [321]:
def calc_ind_UF(base_model_result, ind_result, calc_schema='test_sch'):
    """ 
    ** calculation is for ONE impact category (e.g., GWP)
    
    params:
    - base_model_result: LCIA result of base model (e.g., GWP)
    - ind_result: LCIA result of an individual instance of variation, (e.g., GWP)
    - calc_schema: method for calculting uncertianty factor for individual instances of variation
        - test_sch: based on "uncertainty methodology_v5"
    """
    
    ## calculate the UF for the instance of variation
    if calc_schema == 'test_sch':
        ind_UF = (ind_result - base_model_result) / base_model_result
    else:
        pass
    
    ## return the UF result dict
    return ind_UF
        
        
    
    

#### 1.6 create a function to calculate (1) the overall UF for a given uncertainty group and (2) final UF derived from all uncertainty groups
- overall uncertainty factor of a given uncertainty group is calculated by aggregating (e.g., summation) of the UF of individual instances of variation

In [322]:
def calc_final_UF(ind_UF_all_method_dict, mapping_dict, lcia_method_lst, agg_ind_method='sum', final_der_method='rss'):
    """
    params:
    - ind_UF_all_method_dict: store UF (by each method) of individual instances of variation, 
        {"scenario_x1": {'lcia_method1': UF_1, ..., 'lcia_methodX': UF_X},
         ...,
         "scenario_xi": {'lcia_method1': UF_1, ..., 'lcia_methodX': UF_X}
        } 
    - agg_ind_method: method for aggregate the UF of individual instances of variation into one overall UF for a given uncertainty group
        - 'sum': sum the UFs of individual instances of variation.
    - final_der_method: method for deriving final UF from all uncertainty groups
        - 'root sum square': root sum square of UFs of each uncertainty group
    - mapping_dict: map UFs of individual instances of variation to respetive uncertainty groups, {"uncertainty_group_1": ["scenario_x9",..]}
    """
    
    ## prepare variables
    group_UF_all_method_dict = {} # store UF of each uncertainty group 
                                  # {"uncertainty_group_1": {'lcia_method1': g_UF_1, ..., 'lcia_methodX': g_UF_X},
                                  # ..., 
                                  # "uncertainty_group_j": {'lcia_method1': g_UF_1, ..., 'lcia_methodX': g_UF_X) 
                                  # }

    ## aggregate individual instances of variation under corresponding uncertainty group
    for group_name, sc_lst in mapping_dict.items():
        if agg_ind_method == 'sum':
            group_UF_all_method_dict[group_name] = {}
            # loop over all methods 
            for method in lcia_method_lst:
                 group_UF_all_method_dict[group_name][method] = sum([ind_UF_all_method_dict[sc][method] for sc in sc_lst])
        else:
            pass
    
    ## derive final UF from group UFs
    final_UF_all_method_dict = {} # {
                                  # 'method1': f_UF1,
                                  # ...,
                                  # 'methodi': f_UFi
                                  # }
    
    if final_der_method == 'rss':
        for method in lcia_method_lst:
            final_UF_all_method_dict[method] = math.sqrt(sum([g_UF_dict[method]**2 for g_UF_dict in group_UF_all_method_dict.values()]))
    else:
        pass
    
    ## return final UF
    return final_UF_all_method_dict
        
            
    
    
    

#### 1.7 create a wrapper function to run LCA and UF calculations

In [323]:
def wrapper_LCA_UF(base_model_N_amt, subt_exc_dict, update_exc_dict, LCIA_methods, mapping_dict, **kwargs):
    """
    params:
    - base_model_N_amt: a tuple contains base model ('db', act_code) and corresponding amount
    - subt_exc_dict: a dict contains original exchanges whose LCIA results need to be subtracted from that of base model,
        {"scenario_x1": {'layer0': [{'market for electricity, medium voltage_uuid':0.5}, ..., {'limestone, packed_uuid'':1.1}],
                ...
               'layer5': [{'market for electricity, medium voltage_uuid'':0.7}, ..., {'hydrogen production, steam reforming_uuid'':0.1}]
                },
         ...
         "scenario_xi": {'layer0': [{'market for electricity, medium voltage_uuid':0.1}, ..., {'limestone, packed_uuid'':0.1}],
                ...
               'layer4': [{'market for electricity, medium voltage_uuid'':1.2}, ..., {'hydrogen production, steam reforming_uuid'':1.1}]
                }
        }
        ** this subtraction yields a 'intermediate LCIA result'
    - update_exc_dict: a dict contrains exchanges whose LCIA results need to be added to that of intermediate LCIA result,
        {"scenario_x1": {
            ('db', new_input_code1'): amt1, 
            ('db', new_input_code2'): amt2, 
            ...
            ('db', new_input_codeN'): amtN, 
            }
            ...
         "scenario_xi": {
            ('db', new_input_code1'): amt1, 
            ('db', new_input_code2'): amt2, 
            ...
            ('db', new_input_codeN'): amtN, 
            }
        }
        ** the 'scenario' keys should be the SAME between subt_exc_dict and this dict
    - LCIA_methods: a list of LCIA methods
    - mapping_dict: map UFs of individual instances of variation to respetive uncertainty groups, {"uncertainty_group_1": ["scenario_x9",..]}
    """
    
    ## parse kwargs
    if 'UUID_only' in kwargs:
        UUID_only_flg = kwargs['UUID_only']
    else:
        UUID_only_flg = False
    
    ## prepare output variables
    final_LCIA_dict = {} # {"scenario_x1": {'lcia_method1': final_result1, ..., 'lcia_methodX': final_resultX},
                         #  ...
                         #  "scenario_xi": {'lcia_method1': final_result1, ..., 'lcia_methodX': final_resultX},
                         # }
    ind_UF_all_method_dict = {} # {"scenario_x1": {'lcia_method1': UF_1, ..., 'lcia_methodX': UF_X},
                                #  ...
                                #  "scenario_xi": {'lcia_method1': UF_1, ..., 'lcia_methodX': UF_X},
                                # }
    
    ## calculate LCIA results for instances of variation
    base_model_act = base_model_N_amt # a tuple contains base model ('db', act_code) and corresponding amount
    for sc in subt_exc_dict.keys(): # 'subt_exc_dict' and 'update_exc_dict' MUST have SAME KEYs
        # perform subtraction
        base_results, intermdiate_results = subt_lcia(base_act=base_model_act, inputs_to_subt_dict=subt_exc_dict[sc], 
                                                lcia_method_lst=lcia_methods, UUID_only_flg=UUID_only_flg)
           
        # print out base results
        print(f"Base model results are: {base_results}")
    
        # update the model and store the final LCIA result
        updated_lcia_results = update_lcia(intermdiate_results, update_exc_dict[sc])
        final_LCIA_dict[sc] = updated_lcia_results
        
        # calculate UF of individual instances of variation for all LCIA methods
        ind_UF_all_method_dict[sc] = {}
        
        for method in base_results.keys(): # {'lcia_method1': lcia_result1, 'lcia_method2': lcia_result2, ...}
            ind_UF = calc_ind_UF(base_results[method], final_LCIA_dict[sc][method], calc_schema='test_sch')
            ind_UF_all_method_dict[sc][method] = ind_UF
            
    ## derive group UF and the final UF for all LCIA methods    
    final_UF_dict = calc_final_UF(ind_UF_all_method_dict, mapping_dict, LCIA_methods, agg_ind_method='sum', final_der_method='rss')
                                                                                                                 
    ## return final UF and LCIA results for all LCIA methods
    return final_UF_dict, final_LCIA_dict
    

#### 1.8 create helper function to use the recursion algorithm

In [324]:
def recur_helper(base_model,base_model_act,info_tuple,tmp_act_dict,db,act_code_map, **kwargs):
    """ This helper function uses recurrsion algorithm to traverse the base model
    - base_model: the LCA model of interest (e.g., 'flat glass production, uncoated, RoW')
    - base_model_act: the activity refering to the LCA model
    - info_tuple: a tuple of information used to fetch the correct amt for the act of interest (e.g., (layer_from,parent,act_from,amt_from))
    - tmp_act_dict: a dict to storage the pair of tuple of information (key) and corresponding amt (value)
        - this dict is updated (e.g., keeps growing) each time 'recur_helper' is called
    - db: database where base_model resides (e.g., ecoinvent 3.5 cutoff)
    - act_code_map: a dict to record the act and its code {act: act['code']}
    """
    # parse the information tuple
    layer,parent,act_,amt_ = info_tuple
        
    # parse activity name and location
    tmp_proc = act_.split("|")[0].strip() # e.g., get process name from the the string of act_from 
    tmp_loc = act_.split("|")[-1].strip().split("--")[-1].strip() # e.g., get 'RoW' from '... | Cutoff, U --RoW'   
    if tmp_loc == 'all_loc':
        all_loc_label = True # this will be used to trigger sum() to aggregate the amt later
        tmp_loc_lst = [act['location'] for act in db if tmp_proc in act['name']]
        tmp_search_lst = [(tmp_proc, el) for el in tmp_loc_lst]
        print("for all_loc, this is the list of locations available: ", tmp_loc_lst, "\n")
    else:
        tmp_search_lst = [(tmp_proc, tmp_loc)]

    """ check what kind of parsing work ('from' or 'to') is expected """
    if not ('info_from_dict_amt' in kwargs) and not ('info_from_dict_parent_amt' in kwargs): # if user is trying to parse the 'from' side  
        # prepare the 'info_from_dict': used to store information to be used in parsing corresponding 'to' side
        info_from_dict_amt = {} # {(layer_from,parent,tmp_proc): amt} # when 'all_loc', amt are summed
        info_from_dict_parent_amt = {} # {(layer_from,parent,tmp_proc): parent_amt} # when 'all_loc', all parent_amt are the same
        parent_amt_dict = {} # {(layer_from,parent,tmp_proc,tmp_loc): parent_amt}

        # fetch correct amt for the act of interest (i.e., to be subtracted or updated)          
        for (_proc, _loc) in tmp_search_lst:
            # record the act and its code
            tmp_proc_loc = _proc + " | " + _loc
            act_code_map[tmp_proc_loc] = [act['code'] for act in db if _proc in act['name'] 
                                      and _loc in act['location']][0]

            # reset tmp_amt, otherwise it may get carried over
            tmp_amt = None

            if (layer,parent,_proc,_loc) not in tmp_act_dict.keys(): 
                # using the recursion function                     
                tmp_glo_store_dict = defaultdict(list)
                tmp_glo_relation_dict = defaultdict(list)
                #print(f"temp search list is: {tmp_search_lst}", "\n")

                # create a 'local' temp search list
                local_tmp_search_lst = [(_proc, _loc)]

                recur_dig(base_model,1,local_tmp_search_lst,layer+1,
                          tmp_glo_store_dict,tmp_glo_relation_dict,base_model_act['name'],1) # 1st '1' is the amt for reference product, 
                                                                                             # 2nd '1' is the parent_amt (which happpen to be the same amt for reference prod)
                tmp_glo_store_dict_agg = recur_agg(tmp_glo_store_dict, agg_by_name_only=False)
                #print(f"temp recurrsion result dict is {tmp_glo_store_dict}", "\n")
                #print(f"temp recurrsion (agg) result dict is {tmp_glo_store_dict_agg}","\n")
                #print(f"temp resurrsion relatinoships are {tmp_glo_relation_dict}", "\n")

                # identify the correct default value that corresponds the parent act of interest
                for relationship in tmp_glo_relation_dict[0]: # loop over the list of relationships at the layer where act of interes is identified 
                                                              # (as recurrsion keeps reducing depth until you find the act of interest, which means depth is zero)
                    if parent.split("|")[0].strip() in relationship and parent.split("|")[1].strip() in relationship:
                        tmp_idx = tmp_glo_relation_dict[0].index(relationship) # get the index that corresponds to the correct amt in 'tmp_glo_store_dict[0]'
                        tmp_amt_dict = tmp_glo_store_dict[0][tmp_idx] # do NOT use 'tmp_glo_store_dict_agg[0]' 
                                                                      # (b/c same act of interest from different parents are aggregated already in that dict)
                        # fetch the parent_amt
                        tmp_parent_amt = float(relationship.split("|")[-1].strip())

                        # check if user wants 'default' or specified amt
                        if amt_ == 'default':
                            tmp_amt = list(tmp_amt_dict.values())[0] # this gives the actual amt
                            tmp_amt_current_layer_only = tmp_amt / tmp_parent_amt
                        else:
                            tmp_amt = tmp_parent_amt * amt_
                            tmp_amt_current_layer_only = amt_

                # add fetched/calculated value to the dict
                if not(tmp_amt == None): 
                    tmp_act_dict[(layer,parent,_proc,_loc)] = tmp_amt # this dict keeps update itself (i.e., global variable) everytime 'recur_helpler' is called
                    parent_amt_dict[(layer,parent,_proc,_loc)] = tmp_parent_amt # this dict gets reset everytime (i.e., local variable)'recur_helpler' is called

            else: # this assumes that there will be only ONE update on the SAME parent-child relationship at the SAME layer
                tmp_amt = tmp_act_dict[(layer,parent,_proc,_loc)]
                """ here we assume info_from_dict has been created and returned when corresponding (key,val) was created """ 

        # sum the individual amt_loc and return 'info_from_dict'
        for _, tmp_amt in tmp_act_dict.items():
            if tmp_loc == 'all_loc': # aggregate the actual amt at tmp_proc lvl
                info_from_dict_amt[info_tuple[:-1]] = info_from_dict_amt.get(info_tuple[:-1],0) + tmp_amt # in this case directly use (layer, parent, act (loc=all_loc))
            else:
                info_from_dict_amt[info_tuple[:-1]] = tmp_amt

        for _, parent_amt in parent_amt_dict.items():
            info_from_dict_parent_amt[info_tuple[:-1]] = parent_amt # multiple actitives (same name, different locations) should have the same parent_amt 

        return info_from_dict_amt, info_from_dict_parent_amt

    else: # if the user is parsing the 'to' side (i.e., when 'info_from_dict' is provided)
        # fetch correct amt for the act of interest (i.e., to be subtracted or updated)          
        for (_proc, _loc) in tmp_search_lst:
            # record the act and its code
            tmp_proc_loc = _proc + " | " + _loc
            act_code_map[tmp_proc_loc] = [act['code'] for act in db if _proc in act['name'] 
                                      and _loc in act['location']][0]

            # reset tmp_amt, otherwise it may get carried over
            tmp_amt = None

            if (layer,parent,_proc,_loc) not in tmp_act_dict.keys(): 
                # using the recursion function                     
                tmp_glo_store_dict = defaultdict(list)
                tmp_glo_relation_dict = defaultdict(list)
                #print(f"temp search list is: {tmp_search_lst}", "\n")

                # create a 'local' temp search list
                local_tmp_search_lst = [(_proc, _loc)]

                recur_dig(base_model,1,local_tmp_search_lst,layer+1,
                          tmp_glo_store_dict,tmp_glo_relation_dict,base_model_act['name'],1) # 1st '1' is the amt for reference product, 
                                                                                             # 2nd '1' is the parent_amt (which happpen to be the same amt for reference prod)
                tmp_glo_store_dict_agg = recur_agg(tmp_glo_store_dict, agg_by_name_only=False)
                #print(f"temp recurrsion result dict is {tmp_glo_store_dict}", "\n")
                #print(f"temp recurrsion (agg) result dict is {tmp_glo_store_dict_agg}","\n")
                #print(f"temp resurrsion relatinoships are {tmp_glo_relation_dict}", "\n")
                
                # if new parent-child relationship is created (e.g., swap crushed lime for packed lime in flat glass production)
                # 'tmp_glo_relation_dict' will be empty
                if len(tmp_glo_relation_dict) == 0:
                    if amt_ == 'no_change':
                        for k, amt in kwargs['info_from_dict_amt'].items(): # k = (layer_from,parent,tmp_proc)
                            if k[:-1] == (layer,parent): # identify the correct parent (both "From" and "To" should at the same layer (i.e., same parent))
                                tmp_act_dict[(layer,parent,_proc,_loc)] = amt          
                    else:
                        tmp_act_dict[(layer,parent,_proc,_loc)] = amt_
                    #tmp_glo_relation_dict[0] = _proc + " | " + _loc + " --> " + parent + " | " + "THIS IS CREATED"
                
                # identify the correct default value that corresponds the parent act of interest
                for relationship in tmp_glo_relation_dict[0]: # loop over the list of relationships at the layer where act of interes is identified 
                                                              # (as recurrsion keeps reducing depth until you find the act of interest, which means depth is zero)
                    if parent.split("|")[0].strip() in relationship and parent.split("|")[1].strip() in relationship:
                        tmp_idx = tmp_glo_relation_dict[0].index(relationship) # get the index that corresponds to the correct amt in 'tmp_glo_store_dict[0]'
                        tmp_amt_dict = tmp_glo_store_dict[0][tmp_idx] # do NOT use 'tmp_glo_store_dict_agg[0]' 
                                                                      # (b/c same act of interest from different parents are aggregated already in that dict)
                        # fetch the parent_amt
                        tmp_parent_amt = float(relationship.split("|")[-1].strip())

                        # check if user wants 'default' or specified amt
                        if amt_ == 'default':
                            tmp_amt = list(tmp_amt_dict.values())[0] # this gives the actual amt
                            tmp_amt_current_layer_only = tmp_amt / tmp_parent_amt
                        elif amt_ == 'no_change':
                            for k, amt in kwargs['info_from_dict_amt'].items(): # k = (layer_from,parent,tmp_proc)
                                if k[:-1] == (layer,parent): # identify the correct parent (both "From" and "To" should at the same layer (i.e., same parent))
                                    tmp_act_dict[(layer,parent,_proc,_loc)] = amt                            
                        else:
                            tmp_amt = tmp_parent_amt * amt_
                            tmp_amt_current_layer_only = amt_

                # add fetched/calculated value to the dict
                if not(tmp_amt == None): 
                    tmp_act_dict[(layer,parent,_proc,_loc)] = tmp_amt # this dict keeps update itself (i.e., global variable) everytime 'recur_helpler' is called
                    parent_amt_dict[(layer,parent,_proc,_loc)] = tmp_parent_amt # this dict gets reset everytime (i.e., local variable)'recur_helpler' is called

            else: # this assumes that there will be only ONE update on the SAME parent-child relationship at the SAME layer
                tmp_amt = tmp_act_dict[(layer,parent,_proc,_loc)]
                """ here we assume info_from_dict has been created and returned when corresponding (key,val) was created """ 



#### 1.9 create a scenario parser to automate the preparation dicts for LCA and UF calculations

In [325]:
def scenario_parser(scenario_dict, base_model_info, db):
    """ this function parses the scenario dict and output the variables for the Wrapper function 
    - scenario_dict
    - base_model_info: e.g., ('ecoinvent 3.5 cutoff', unit_proc_name, unit_proc_loc)
    
    """
    
    ## initialize output variables (mapping_dict will be specified MANUALLY when UF is calculated)
    subt_exc_dict, update_exc_dict = {}, {}
    
    ## create base model
    db_name, base_model_proc_name, base_model_loc = base_model_info
    base_model_act = [act for act in db if base_model_proc_name in act['name']
                     and base_model_loc in act['location']][0]
    print(base_model_proc_name +  " SHOULD BE " + base_model_act['name'] + "\n")
    base_model = (db_name, base_model_act['code'])
    
    ## populate output variables
    for scenario_name, subdict in scenario_dict.items():
        # create a dict to store the amt of act 
        tmp_act_from_dict, tmp_act_to_dict = {}, {} # {(layer, parent, act, loc): amt}
        act_code_map_from, act_code_map_to = {}, {} # {tmp_proc_loc: act_code}
            
        for (layer_from,parent_from,act_from,amt_from), (layer_to,parent_to,act_to,amt_to) in subdict.items():
            """ prepare intermediate dict for subt_exc_dict """
            tmp_info_tuple = (layer_from,parent_from,act_from,amt_from)
            info_from_dict_amt, info_from_dict_parent_amt = recur_helper(base_model,base_model_act,
                                                                         tmp_info_tuple,tmp_act_from_dict,db,act_code_map_from)
            print(f"This is {scenario_name}, corresponding parsed FROM dict is: ", "\n", tmp_act_from_dict,"\n")
            print(f"collected amt info from is: {info_from_dict_amt}", "\n")
            print(f"collected parent_amt info from is: {info_from_dict_parent_amt}", "\n")
            
            """ intermediate dict for prepare update_exc_dict """
            tmp_info_tuple = (layer_to,parent_to,act_to,amt_to) # [CAUTION] 'layer_to' and 'parent_to' MUST be the same as 'layer_from' and 'parent_from'
            recur_helper(base_model,base_model_act,tmp_info_tuple,tmp_act_to_dict,db,act_code_map_to,
                        info_from_dict_amt=info_from_dict_amt, info_from_dict_parent_amt=info_from_dict_parent_amt)
            print(f"This is {scenario_name}, corresponding parsed TO dict is: ", "\n", tmp_act_to_dict,"\n")
        
        """ format subt_exc_dict """
        tmp_subdict = defaultdict(list)
        for (layer, parent, act, loc), amt in tmp_act_from_dict.items():
            # parse the act code
            tmp_proc_loc_combo = act + " | " + loc # this MUST match the format of 'tmp_proc_loc' (key) in act_code_map
            tmp_subdict[layer].append({act_code_map_from[tmp_proc_loc_combo]: amt})
        
        subt_exc_dict[scenario_name] = tmp_subdict
        #print(f"activities to be subtracted is {subt_exc_dict}", "\n")
        
        """ format update_exc_dict """
        tmp_subdict = defaultdict(list)
        for (layer, parent, act, loc), amt in tmp_act_to_dict.items():
            # parse the act code
            tmp_proc_loc_combo = act + " | " + loc # this MUST match the format of 'tmp_proc_loc' (key) in act_code_map
            tmp_FU_tmp = (db_name,act_code_map_to[tmp_proc_loc_combo])
            tmp_subdict[layer].append({tmp_FU_tmp: amt})
        
        update_exc_dict[scenario_name] = tmp_subdict #{scenario: {layer: [{(db, act[code]): amt}, {(db, act[code]): amt}...]} }
        #print(f"activities to be UPDATED is {update_exc_dict}", "\n")

    return subt_exc_dict, update_exc_dict
    


### 2. set up project

In [326]:
bw.projects.set_current("EPD uncertainty analysis")
bw.projects.current

'EPD uncertainty analysis'

In [5]:
bw.bw2setup()

Biosphere database already present!!! No setup is needed


#### import Ecoinvent

In [8]:
#create an object to import database
ei35_path=os.getcwd()
ei35_cutoff=bw.SingleOutputEcospold2Importer(
    os.path.join(ei35_path,'ecoinvent 3.5_cutoff_ecoSpold02/datasets'),
    "ecoinvent 3.5 cutoff") #"/Users/Qingshi/Downloads/ecoinvent 3.5_cutoff_ecoSpold02/datasets"
ei35_cutoff.apply_strategies()
ei35_cutoff.statistics()

Extracting XML data from 16022 datasets
Extracted 16022 datasets in 71.33 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: 

(16022, 544735, 0)

In [10]:
#to actually write the database into hard disc
ei35_cutoff.write_database()

NameError: name 'ei35_cutoff' is not defined

In [327]:
db=bw.Database("ecoinvent 3.5 cutoff") #to create a database object

### 3. Example: flat glass production

In [328]:
""" identify the unit process for update"""
flat_glass_uncoated_RoW=db.search('flat glass')[3]
flat_glass_uncoated_RoW
flat_glass_uncoated_RoW['code']
 

' identify the unit process for update'

'flat glass production, uncoated' (kilogram, RoW, None)

'235dd47b87b96827f58acd84618f9505'

In [7]:
RECIPE_methods=[method for method in list(bw.methods) if 'ReCiPe' in method[0]]
RECIPE_methods

[('ReCiPe Endpoint (E,A)',
  'ecosystem quality',
  'agricultural land occupation'),
 ('ReCiPe Endpoint (E,A)', 'ecosystem quality', 'climate change, ecosystems'),
 ('ReCiPe Endpoint (E,A)', 'ecosystem quality', 'freshwater ecotoxicity'),
 ('ReCiPe Endpoint (E,A)', 'ecosystem quality', 'freshwater eutrophication'),
 ('ReCiPe Endpoint (E,A)', 'ecosystem quality', 'marine ecotoxicity'),
 ('ReCiPe Endpoint (E,A)', 'ecosystem quality', 'natural land transformation'),
 ('ReCiPe Endpoint (E,A)', 'ecosystem quality', 'terrestrial acidification'),
 ('ReCiPe Endpoint (E,A)', 'ecosystem quality', 'terrestrial ecotoxicity'),
 ('ReCiPe Endpoint (E,A)', 'ecosystem quality', 'total'),
 ('ReCiPe Endpoint (E,A)', 'ecosystem quality', 'urban land occupation'),
 ('ReCiPe Endpoint (E,A)', 'human health', 'climate change, human health'),
 ('ReCiPe Endpoint (E,A)', 'human health', 'human toxicity'),
 ('ReCiPe Endpoint (E,A)', 'human health', 'ionising radiation'),
 ('ReCiPe Endpoint (E,A)', 'human health',

In [20]:
bw.methods

Methods dictionary with 850 objects, including:
	('CML 2001', 'acidification potential', 'average European')
	('CML 2001', 'acidification potential', 'generic')
	('CML 2001', 'climate change', 'GWP 100a')
	('CML 2001', 'climate change', 'GWP 20a')
	('CML 2001', 'climate change', 'GWP 500a')
	('CML 2001', 'climate change', 'lower limit of net GWP')
	('CML 2001', 'climate change', 'upper limit of net GWP')
	('CML 2001', 'eutrophication potential', 'average European')
	('CML 2001', 'eutrophication potential', 'generic')
	('CML 2001', 'freshwater aquatic ecotoxicity', 'FAETP 100a')
Use `list(this object)` to get the complete list.

#### *** Flat glass demo ***

In [44]:
""" prepare input variables """
# scenario dict: { scenario1: {(layer, act_from, amt): (layer, act_to, amt), (layer, act_from, amt): (layer, act_to, amt)},
#                  scenario2: {(layer, act_from, amt): (layer, act_to, amt), (layer, act_from, amt): (layer, act_to, amt)}, ..}
scenario_dict={
    'change soda ash provider at facility | GLO->RER':
        {
            (0,'market for soda ash, light, crystalline, heptahydrate | soda ash, light, crystalline, heptahydrate | Cutoff, U -GLO','default'): (0,'soda production, solvay process | soda ash, light, crystalline, heptahydrate | Cutoff, U -RER','default')
        },
    'change soda ash provider at facility | GLO->RoW': 
        {
            (0,'market for soda ash, light, crystalline, heptahydrate | soda ash, light, crystalline, heptahydrate | Cutoff, U -GLO','default'): (0,'soda production, solvay process | soda ash, light, crystalline, heptahydrate | Cutoff, U -RoW','default')
        },
    'change soda ash provider at facility | light->dense': 
        {
            (0,'market for soda ash, light, crystalline, heptahydrate | soda ash, light, crystalline, heptahydrate | Cutoff, U -GLO','default'): (0,'market for soda ash, dense | soda ash, dense | Cutoff, U -GLO','default')
        },
    'change soda ash amt (foreground) | default->zero': 
        {
            (0,'market for soda ash, light, crystalline, heptahydrate | soda ash, light, crystalline, heptahydrate | Cutoff, U -GLO','default'): (0,'market for soda ash, light, crystalline, heptahydrate | soda ash, light, crystalline, heptahydrate | Cutoff, U -GLO',0.0)
        }, 
    'change soda ash amt (foreground) | default->new_val': 
        {
            (0,'market for soda ash, light, crystalline, heptahydrate | soda ash, light, crystalline, heptahydrate | Cutoff, U -GLO','default'): (0,'market for soda ash, light, crystalline, heptahydrate | soda ash, light, crystalline, heptahydrate | Cutoff, U -GLO',0.05)
        }, 
    'change elect provider at facility (foreground, layer0) | Cutoff, U -all_loc->IN':
        {
            (0,'market group for electricity, medium voltage | Cutoff, U -all_loc', 'default'): (0,'market group for electricity, medium voltage | electricity, medium voltage | Cutoff, U -IN', 'default')
        },
    'change elect provider at facility (foreground, layer0) | Cutoff, U -all_loc->DE':
        {
            (0,'market group for electricity, medium voltage | Cutoff, U -all_loc', 'default'): (0,'market group for electricity, medium voltage | electricity, medium voltage | Cutoff, U -DE', 'default')
        },
    'change elect provider at facility (foreground, layer0) | Cutoff, U -all_loc->RER':
        {
            (0,'market group for electricity, medium voltage | Cutoff, U -all_loc', 'default'): (0,'market group for electricity, medium voltage | electricity, medium voltage | Cutoff, U -RER', 'default')
        },
    'change elect provider at facility (foreground, layer0) | Cutoff, U -all_loc->US':
        {
            (0,'market group for electricity, medium voltage | Cutoff, U -all_loc', 'default'): (0,'market group for electricity, medium voltage | electricity, medium voltage | Cutoff, U -US', 'default')
        },
    'change elect provider at facility (foreground, layer0) | Cutoff, U -all_loc->CH':
        {
            (0,'market group for electricity, medium voltage | Cutoff, U -all_loc', 'default'): (0,'market group for electricity, medium voltage | electricity, medium voltage | Cutoff, U -CH', 'default')
        },
    'change elect provider at facility (foreground, layer0) | Cutoff, U -all_loc->GB':
        {
            (0,'market group for electricity, medium voltage | Cutoff, U -all_loc', 'default'): (0,'market group for electricity, medium voltage | electricity, medium voltage | Cutoff, U -GB', 'default')
        },
    'change elect provider at facility (foreground, layer0) | Cutoff, U -all_loc->PL':
        {
            (0,'market group for electricity, medium voltage | Cutoff, U -all_loc', 'default'): (0,'market group for electricity, medium voltage | electricity, medium voltage | Cutoff, U -PL', 'default')
        },
    'change elect provider at facility (foreground, layer0) | Cutoff, U -all_loc->FR':
        {
            (0,'market group for electricity, medium voltage | Cutoff, U -all_loc', 'default'): (0,'market group for electricity, medium voltage | electricity, medium voltage | Cutoff, U -FR', 'default')
        },
    'Change heavy fuel oil provider at facility (foreground, layer0) | RoW->RER':
        {
            (0,'market for heavy fuel oil | heavy fuel oil | Cutoff, U -RoW', 'default'): (0,'market for heavy fuel oil | heavy fuel oil | Cutoff, U -RER', 'default')
        },
    'Change heavy fuel oil provider at facility (foreground, layer0) | RoW->CH':
        {
            (0,'market for heavy fuel oil | heavy fuel oil | Cutoff, U -RoW', 'default'): (0,'market for heavy fuel oil | heavy fuel oil | Cutoff, U -CH', 'default')
        },
    'Change heavy fuel oil amt at facility (foreground, layer0) | default->zero':
        {
            (0,'market for heavy fuel oil | heavy fuel oil | Cutoff, U -RoW', 'default'): (0,'market for heavy fuel oil | heavy fuel oil | Cutoff, U -RoW', 0.0)
        },
}




' prepare input variables '

In [329]:
""" test scenario parser """
base_model_info_test = ('ecoinvent 3.5 cutoff', 'flat glass production, uncoated', 'RoW')
scenario_dict_test = {
    "change transport for lime mrk RoW":
    {
        (1,'market for lime, packed | RoW', 'market for transport, freight train | transport, freight train | Cutoff, U --all_loc','default'): (1,'market for lime, packed | RoW','market for transport, freight train | transport, freight train | Cutoff, U --RoW','no_change'),
        
    },
    "change lime RoW":
    {
        (0,'flat glass production, uncoated | RoW','market for lime, packed | lime, packed | Cutoff, U --RoW','default'): (0,'flat glass production, uncoated | RoW','limestone production, crushed, for mill | limestone, crushed, for mill | Cutoff, U --RoW', 'no_change'),
    },
    "change elect NRA":
    {
        (0,'flat glass production, uncoated | RoW','market group for electricity, medium voltage | electricity, medium voltage | Cutoff, U --RNA','default'): (0,'flat glass production, uncoated | RoW','market for electricity, medium voltage | electricity, medium voltage | Cutoff, U --US-MRO', 1.5),        
    },

}

#        (1,'market for lime, packed | RoW', 'market for transport, freight train | transport, freight train | Cutoff, U-RoW','default'): (1,'market for lime, packed | RoW','market for transport, freight train | transport, freight train | Cutoff, U-RoW',0.5)


subt_exc_dict, update_exc_dict = scenario_parser(scenario_dict_test, base_model_info_test, db)


' test scenario parser '

flat glass production, uncoated SHOULD BE flat glass production, uncoated

for all_loc, this is the list of locations available:  ['Europe without Switzerland', 'CH', 'RoW', 'US', 'CN'] 

This is change transport for lime mrk RoW, corresponding parsed FROM dict is:  
 {(1, 'market for lime, packed | RoW', 'market for transport, freight train', 'RoW'): 0.00483999964788576, (1, 'market for lime, packed | RoW', 'market for transport, freight train', 'US'): 2.0568611226063e-10, (1, 'market for lime, packed | RoW', 'market for transport, freight train', 'CN'): 1.464281321853528e-10} 

collected amt info from is: {(1, 'market for lime, packed | RoW', 'market for transport, freight train | transport, freight train | Cutoff, U --all_loc'): 0.004840000000000004} 

collected parent_amt info from is: {(1, 'market for lime, packed | RoW', 'market for transport, freight train | transport, freight train | Cutoff, U --all_loc'): 0.4} 

This is change transport for lime mrk RoW, corresponding parsed T

In [330]:
""" test compatibility with calculation functions"""
## prepare variables
db_name = 'ecoinvent 3.5 cutoff'
base_model_N_amt = ((db_name, flat_glass_uncoated_RoW['code']), 1)
lcia_methods = [('CML 2001', 'climate change', 'GWP 100a')]

mapping_dict = {
    "uncertainty group ONE": ["change elect NRA", "change lime RoW"],
    "uncertainty group TWO": ["change transport for lime mrk RoW"]
}

## run wrapper
final_UF_dict, final_LCA_results_dict = wrapper_LCA_UF(base_model_N_amt=base_model_N_amt, subt_exc_dict=subt_exc_dict, 
                               update_exc_dict=update_exc_dict, LCIA_methods=lcia_methods, mapping_dict=mapping_dict, UUID_only=True)


# show 'final_UF_dict'
print(final_UF_dict, "\n")

# show final_LCA_results_dict
print(final_LCA_results_dict, "\n")



' test compatibility with calculation functions'

Base model results are: {('CML 2001', 'climate change', 'GWP 100a'): 1.0288643274790947}
Base model results are: {('CML 2001', 'climate change', 'GWP 100a'): 1.0288643274790947}
Base model results are: {('CML 2001', 'climate change', 'GWP 100a'): 1.0288643274790947}
{('CML 2001', 'climate change', 'GWP 100a'): 1.0602665300305243} 

{'change transport for lime mrk RoW': {('CML 2001', 'climate change', 'GWP 100a'): 1.0288643274777636}, 'change lime RoW': {('CML 2001', 'climate change', 'GWP 100a'): 1.0038667397806043}, 'change elect NRA': {('CML 2001', 'climate change', 'GWP 100a'): 2.144732325546034}} 

